<a href="https://colab.research.google.com/github/ConstructoDestructo/Computer_Architecture_Project/blob/main/NHANES_DATA_SIMPLE_REFORMATTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
NHANES DATA PREPARATION - WITH LAB VALUES
==========================================

This version includes BOTH:
1. Basic variables (for public screening)
2. Lab values (for clinical assessment)

Use this for calibrating BOTH modes of your tool!
"""

import pandas as pd
import numpy as np

print("="*80)
print("NHANES DATA PREPARATION - WITH LAB VALUES")
print("="*80)

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================

DATA_PATH = 'nhanes_patient_flattened_enhanced.csv'

print(f"\nLoading data from: {DATA_PATH}")
df = pd.read_csv(DATA_PATH, low_memory=False)
print(f"✓ Loaded {len(df):,} patient records")

# ============================================================================
# STEP 2: MAP BASIC COLUMNS (Same as before)
# ============================================================================

print("\n" + "="*80)
print("MAPPING BASIC COLUMNS")
print("="*80)

df_clean = pd.DataFrame()

# AGE
if 'DEMO_RIDAGEYR' in df.columns:
    df_clean['age'] = df['DEMO_RIDAGEYR']
    print(f"✓ Age: {df_clean['age'].notna().sum():,} values")

# SEX
if 'DEMO_RIAGENDR' in df.columns:
    df_clean['sex'] = df['DEMO_RIAGENDR'].map({1: 'Male', 2: 'Female'})
    print(f"✓ Sex: {df_clean['sex'].notna().sum():,} values")

# BMI
bmi_columns = [col for col in df.columns if 'BMXBMI' in col]
if bmi_columns:
    df_clean['bmi'] = df[bmi_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ BMI: {df_clean['bmi'].notna().sum():,} values")

# WAIST
waist_columns = [col for col in df.columns if 'BMXWAIST' in col]
if waist_columns:
    df_clean['waist'] = df[waist_columns].bfill(axis=1).iloc[:, 0]
    if df_clean['waist'].max() > 100:
        df_clean['waist'] = df_clean['waist'] / 2.54
    print(f"✓ Waist: {df_clean['waist'].notna().sum():,} values")
else:
    df_clean['waist'] = np.nan

# RACE
race_col = 'DEMO_RIDRETH3' if 'DEMO_RIDRETH3' in df.columns else 'DEMO_RIDRETH1'
if race_col in df.columns:
    race_map = {1: 'Hispanic', 2: 'Hispanic', 3: 'Non-Hispanic White',
                4: 'Non-Hispanic Black', 6: 'Non-Hispanic Asian', 7: 'Other/Mixed'}
    df_clean['race'] = df[race_col].map(race_map)
    print(f"✓ Race: {df_clean['race'].notna().sum():,} values")

# ============================================================================
# STEP 3: ADD LAB VALUES (NEW!)
# ============================================================================

print("\n" + "="*80)
print("ADDING LAB VALUES")
print("="*80)

# HbA1c (Glycohemoglobin) - CRITICAL!
hba1c_columns = [col for col in df.columns if 'LBXGH' in col]
if hba1c_columns:
    df_clean['hba1c'] = df[hba1c_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ HbA1c: {df_clean['hba1c'].notna().sum():,} values")
    print(f"  Range: {df_clean['hba1c'].min():.1f} - {df_clean['hba1c'].max():.1f}%")
else:
    print("⚠ HbA1c not found!")
    df_clean['hba1c'] = np.nan

# FASTING GLUCOSE - CRITICAL!
glucose_columns = [col for col in df.columns if 'LBXGLU' in col]
if glucose_columns:
    df_clean['glucose'] = df[glucose_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ Glucose: {df_clean['glucose'].notna().sum():,} values")
    print(f"  Range: {df_clean['glucose'].min():.0f} - {df_clean['glucose'].max():.0f} mg/dL")
else:
    print("⚠ Glucose not found!")
    df_clean['glucose'] = np.nan

# HDL CHOLESTEROL
hdl_columns = [col for col in df.columns if 'LBDHDD' in col or 'LBXHDD' in col]
if hdl_columns:
    df_clean['hdl'] = df[hdl_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ HDL: {df_clean['hdl'].notna().sum():,} values")
    print(f"  Range: {df_clean['hdl'].min():.0f} - {df_clean['hdl'].max():.0f} mg/dL")
else:
    print("⚠ HDL not found!")
    df_clean['hdl'] = np.nan

# LDL CHOLESTEROL
ldl_columns = [col for col in df.columns if 'LBDLDL' in col]
if ldl_columns:
    df_clean['ldl'] = df[ldl_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ LDL: {df_clean['ldl'].notna().sum():,} values")
else:
    print("⚠ LDL not found!")
    df_clean['ldl'] = np.nan

# TRIGLYCERIDES
trig_columns = [col for col in df.columns if 'LBXTR' in col or 'LBXTLG' in col]
if trig_columns:
    df_clean['triglycerides'] = df[trig_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ Triglycerides: {df_clean['triglycerides'].notna().sum():,} values")
else:
    print("⚠ Triglycerides not found!")
    df_clean['triglycerides'] = np.nan

# TOTAL CHOLESTEROL
tc_columns = [col for col in df.columns if 'LBXTC' in col]
if tc_columns:
    df_clean['total_cholesterol'] = df[tc_columns].bfill(axis=1).iloc[:, 0]
    print(f"✓ Total Cholesterol: {df_clean['total_cholesterol'].notna().sum():,} values")
else:
    print("⚠ Total Cholesterol not found!")
    df_clean['total_cholesterol'] = np.nan

# ============================================================================
# STEP 4: CREATE DIABETES LABEL
# ============================================================================

print("\n" + "="*80)
print("CREATING DIABETES LABEL")
print("="*80)

# Questionnaire
diq_columns = [col for col in df.columns if 'DIQ010' in col]
doctor_diagnosis = None
if diq_columns:
    doctor_diagnosis = df[diq_columns].bfill(axis=1).iloc[:, 0]
    doctor_diagnosis = (doctor_diagnosis == 1).astype(float)
    print(f"✓ Doctor diagnosis: {doctor_diagnosis.sum():,.0f} diabetics")

# HbA1c >= 6.5%
hba1c_diabetes = None
if 'hba1c' in df_clean.columns:
    hba1c_diabetes = (df_clean['hba1c'] >= 6.5).astype(float)
    print(f"✓ HbA1c ≥6.5%: {hba1c_diabetes.sum():,.0f} diabetics")

# Glucose >= 126 mg/dL
glucose_diabetes = None
if 'glucose' in df_clean.columns:
    glucose_diabetes = (df_clean['glucose'] >= 126).astype(float)
    print(f"✓ Glucose ≥126: {glucose_diabetes.sum():,.0f} diabetics")

# Combine
diabetes_indicators = []
if doctor_diagnosis is not None:
    diabetes_indicators.append(doctor_diagnosis)
if hba1c_diabetes is not None:
    diabetes_indicators.append(hba1c_diabetes)
if glucose_diabetes is not None:
    diabetes_indicators.append(glucose_diabetes)

if diabetes_indicators:
    diabetes_df = pd.DataFrame(diabetes_indicators).T
    df_clean['diabetes'] = diabetes_df.max(axis=1).fillna(0).astype(int)
    print(f"\n✓ FINAL: {df_clean['diabetes'].sum():,.0f} diabetics ({df_clean['diabetes'].mean()*100:.1f}%)")

# ============================================================================
# STEP 5: CREATE OTHER VARIABLES
# ============================================================================

print("\n" + "="*80)
print("CREATING OTHER VARIABLES")
print("="*80)

# FAMILY HISTORY
fh_columns = [col for col in df.columns if 'DIQ170' in col]
if fh_columns:
    fh_raw = df[fh_columns].bfill(axis=1).iloc[:, 0]
    df_clean['family_history'] = (fh_raw == 1).map({True: 'Yes', False: 'No'})
    print(f"✓ Family History: {(df_clean['family_history']=='Yes').sum():,} positive")
else:
    df_clean['family_history'] = 'No'

# HYPERTENSION
bp_columns = [col for col in df.columns if 'BPQ020' in col]
if bp_columns:
    bp_raw = df[bp_columns].bfill(axis=1).iloc[:, 0]
    df_clean['hypertension'] = (bp_raw == 1).map({True: 'Yes', False: 'No'})
    print(f"✓ Hypertension: {(df_clean['hypertension']=='Yes').sum():,} positive")
else:
    df_clean['hypertension'] = 'No'

# PHYSICAL ACTIVITY
print("✓ Physical Activity: Using simplified categorization")
df_clean['physical_activity'] = 'Moderate'
df_clean.loc[df_clean['age'] >= 65, 'physical_activity'] = 'Light'
df_clean.loc[(df_clean['age'] < 40) & (df_clean['bmi'] < 25), 'physical_activity'] = 'Active'
df_clean.loc[(df_clean['age'] >= 40) & (df_clean['bmi'] >= 30), 'physical_activity'] = 'Sedentary'

# ============================================================================
# STEP 6: CREATE TWO VERSIONS
# ============================================================================

print("\n" + "="*80)
print("DATA CLEANING & FILTERING")
print("="*80)

initial_count = len(df_clean)
print(f"Initial records: {initial_count:,}")

# Remove rows with missing critical BASIC values
df_basic = df_clean.dropna(subset=['age', 'sex', 'bmi']).copy()
print(f"After removing missing age/sex/bmi: {len(df_basic):,} ({len(df_basic)/initial_count*100:.1f}%)")

# Filter to adults
df_basic = df_basic[df_basic['age'] >= 18]
print(f"After filtering to adults (≥18): {len(df_basic):,} ({len(df_basic)/initial_count*100:.1f}%)")

# Remove BMI outliers
df_basic = df_basic[(df_basic['bmi'] >= 15) & (df_basic['bmi'] <= 60)]
print(f"After removing BMI outliers: {len(df_basic):,} ({len(df_basic)/initial_count*100:.1f}%)")

# ============================================================================
# VERSION 1: PUBLIC SCREENING (No Lab Tests Required)
# ============================================================================

print("\n" + "="*80)
print("VERSION 1: PUBLIC SCREENING DATA")
print("="*80)

basic_columns = ['age', 'sex', 'bmi', 'waist', 'race', 'family_history',
                 'hypertension', 'physical_activity', 'diabetes']

df_public = df_basic[basic_columns].copy()
output_public = 'nhanes_PUBLIC_SCREENING.csv'
df_public.to_csv(output_public, index=False)

print(f"✓ Saved: {output_public}")
print(f"✓ Records: {len(df_public):,}")
print(f"✓ Diabetics: {df_public['diabetes'].sum():,} ({df_public['diabetes'].mean()*100:.1f}%)")
print(f"✓ Columns: {list(df_public.columns)}")

# ============================================================================
# VERSION 2: CLINICAL ASSESSMENT (With Lab Values)
# ============================================================================

print("\n" + "="*80)
print("VERSION 2: CLINICAL ASSESSMENT DATA")
print("="*80)

# For clinical version, require lab values
clinical_columns = ['age', 'sex', 'bmi', 'waist', 'race', 'family_history',
                    'hypertension', 'physical_activity',
                    'hba1c', 'glucose', 'hdl', 'ldl', 'triglycerides',
                    'total_cholesterol', 'diabetes']

# Only keep records with at least HbA1c OR glucose
df_clinical = df_basic.copy()
has_lab_data = (df_clinical['hba1c'].notna() | df_clinical['glucose'].notna())
df_clinical = df_clinical[has_lab_data]

print(f"✓ Filtered to records with lab data: {len(df_clinical):,}")

# Keep only clinical columns
df_clinical = df_clinical[clinical_columns].copy()

output_clinical = 'nhanes_CLINICAL_ASSESSMENT.csv'
df_clinical.to_csv(output_clinical, index=False)

print(f"✓ Saved: {output_clinical}")
print(f"✓ Records: {len(df_clinical):,}")
print(f"✓ Diabetics: {df_clinical['diabetes'].sum():,} ({df_clinical['diabetes'].mean()*100:.1f}%)")

# Show lab data availability
print(f"\nLab data availability:")
for col in ['hba1c', 'glucose', 'hdl', 'ldl', 'triglycerides', 'total_cholesterol']:
    if col in df_clinical.columns:
        available = df_clinical[col].notna().sum()
        pct = available / len(df_clinical) * 100
        print(f"  {col:20s}: {available:6,} ({pct:5.1f}%)")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("✅ DATA PREPARATION COMPLETE!")
print("="*80)

print("\nYOU NOW HAVE TWO FILES:")
print("\n1. PUBLIC SCREENING (no labs required):")
print(f"   File: {output_public}")
print(f"   Records: {len(df_public):,}")
print(f"   Use for: Public screening tool calibration")

print("\n2. CLINICAL ASSESSMENT (with labs):")
print(f"   File: {output_clinical}")
print(f"   Records: {len(df_clinical):,}")
print(f"   Use for: Clinical assessment tool calibration")

print("\n" + "="*80)
print("NEXT STEPS:")
print("="*80)
print("1. For PUBLIC screening weights:")
print(f"   DATA_PATH = '{output_public}'")
print("\n2. For CLINICAL assessment weights:")
print(f"   DATA_PATH = '{output_clinical}'")
print("\n3. Run calibration script twice (once for each mode)")
print("="*80)

NHANES DATA PREPARATION - WITH LAB VALUES

Loading data from: nhanes_patient_flattened_enhanced.csv
✓ Loaded 136,803 patient records

MAPPING BASIC COLUMNS
✓ Age: 122,503 values
✓ Sex: 122,503 values
✓ BMI: 100,397 values
✓ Waist: 96,407 values
✓ Race: 60,343 values

ADDING LAB VALUES
✓ HbA1c: 81,404 values
  Range: 2.0 - 18.8%
✓ Glucose: 37,900 values
  Range: 21 - 686 mg/dL
✓ HDL: 72,601 values
  Range: 6 - 226 mg/dL
✓ LDL: 37,714 values
✓ Triglycerides: 38,760 values
✓ Total Cholesterol: 88,317 values

CREATING DIABETES LABEL
✓ Doctor diagnosis: 8,781 diabetics
✓ HbA1c ≥6.5%: 7,481 diabetics
✓ Glucose ≥126: 3,910 diabetics

✓ FINAL: 12,450 diabetics (9.1%)

CREATING OTHER VARIABLES
✓ Family History: 6,743 positive
✓ Hypertension: 24,540 positive
✓ Physical Activity: Using simplified categorization

DATA CLEANING & FILTERING
Initial records: 136,803
After removing missing age/sex/bmi: 87,260 (63.8%)
After filtering to adults (≥18): 56,212 (41.1%)
After removing BMI outliers: 56,052 (